In [1]:
#!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

In [3]:
# driver = webdriver.Chrome()
# driver.get('https://www.wildberries.ru/')

In [4]:
from selenium.webdriver.chrome.options import Options

options = Options()
# запуск на полный экран
options.add_argument('start-maximized')

driver = webdriver.Chrome(options=options)

driver.get('https://www.wildberries.ru/')
time.sleep(3)

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

input = driver.find_element(By.ID, 'searchInput')
input.send_keys('гелевые стельки')
input.send_keys(Keys.ENTER)
time.sleep(2)

Так как процесс парсинга состоит из 3 операций: скролинг (прокрутка страницы), записи данных и пагинации (переход между страницами), выделим эти действия в отдельные функции

функция для скролинга обрабатывает страницу и возвращает список карточек на странице

In [6]:
def scrolling():
    while True:
        # будем ждать пока загрузятся все карточки
        wait = WebDriverWait(driver, timeout=20)
        # ищем карточки
        cards = wait.until(EC.presence_of_all_elements_located((By.XPATH, '//div[@class="product-card-list"]/article')))
        # определяем их количество
        count = len(cards)
        # выполнение кода из JavaScript
        driver.execute_script('window.scrollBy(0,2000)')    # прокрутка вниз на 1000 пикселей
        
        time.sleep(1)
        # вполняем подсчет карточек после скроллинга
        cards = driver.find_elements(By.XPATH, '//div[@class="product-card-list"]/article')
        
        # если кол-во карточек не меняется, выходим из цикла
        if len(cards) == count:
            break
    
    return cards

In [7]:
# функция перехода на следующую страницу
def pagination():
    
    button = driver.find_element(By.XPATH, '//a[@class="pagination-next pagination__next j-next-page"]')
    actions = ActionChains(driver)
    actions.move_to_element(button).click()
    actions.perform()
    

функция добавления значений принимает два аргумента:
* values - список карточек на странице
* list_result - список словарей с информацией о товаре


In [8]:
def value_addition(values, list_result):
    for value in values:
        price = value.find_element(By.CLASS_NAME, 'price__lower-price').text
        name = value.find_element(By.XPATH, './/div/a').get_attribute('aria-label')
        url = value.find_element(By.XPATH, './/div/a').get_attribute('href')
        list_result.append({
            'name': name,
            'url': url,
            'price': price
        })
    return list_result

In [9]:
card_list = []
page = 1

while True:
    # производим скролинг страницы
    cards = scrolling()
    print(f'Скролинг {page} страницы завершен')
    time.sleep(1)
    # делаем проверку на заполненность страницы (столкнулся с проблемой, что появляются страницы с 1 акционным товаром)
    if len(cards) > 1:
        # если список полный, то записываем данные
        value_addition(cards, card_list)
        print(f'Данные {page} страницы записаны')
        time.sleep(0.5)

        page += 1
    else:
        # если нет, перегружаем страницу (одно раза бывает недостаточно)
        driver.refresh()
        print(f'Страницу {page} пришлось обновить')
        time.sleep(2)
        # и завершаем итерацию
        continue
    # переходим на следующую страницу
    try:
        pagination()
        # если кнопка есть, то перешли
        print(f'Перешли на {page} страницу')
        time.sleep(0.5)
    except Exception as e:
        # если кнопки нет, завершаем цикл и выходим
        print(e)
        break



Скролинг 1 страницы завершен
Данные 1 страницы записаны
Перешли на 2 страницу
Скролинг 2 страницы завершен
Данные 2 страницы записаны
Перешли на 3 страницу
Скролинг 3 страницы завершен
Данные 3 страницы записаны
Перешли на 4 страницу
Скролинг 4 страницы завершен
Данные 4 страницы записаны
Перешли на 5 страницу
Скролинг 5 страницы завершен
Данные 5 страницы записаны
Перешли на 6 страницу
Скролинг 6 страницы завершен
Данные 6 страницы записаны
Перешли на 7 страницу
Скролинг 7 страницы завершен
Данные 7 страницы записаны
Перешли на 8 страницу
Скролинг 8 страницы завершен
Данные 8 страницы записаны
Перешли на 9 страницу
Скролинг 9 страницы завершен
Данные 9 страницы записаны
Перешли на 10 страницу
Скролинг 10 страницы завершен
Данные 10 страницы записаны
Перешли на 11 страницу
Скролинг 11 страницы завершен
Данные 11 страницы записаны
Перешли на 12 страницу
Скролинг 12 страницы завершен
Данные 12 страницы записаны
Перешли на 13 страницу
Скролинг 13 страницы завершен
Страницу 13 пришлось обн

In [10]:
driver.quit()

In [11]:
from pprint import pprint
pprint(card_list)

[{'name': 'Стельки для обуви и кроссовок ортопедические мягкие D&K - quality '
          'and comfort',
  'price': '476 ₽',
  'url': 'https://www.wildberries.ru/catalog/209200599/detail.aspx'},
 {'name': 'Стельки гелевые женские и мужские INNORTO',
  'price': '364 ₽',
  'url': 'https://www.wildberries.ru/catalog/213486771/detail.aspx'},
 {'name': 'Гелевые стельки для обуви Healthy Feet',
  'price': '358 ₽',
  'url': 'https://www.wildberries.ru/catalog/155899851/detail.aspx'},
 {'name': 'Стельки гелевые спортивные INNORTO',
  'price': '467 ₽',
  'url': 'https://www.wildberries.ru/catalog/200025262/detail.aspx'},
 {'name': 'Стельки гелевые с валиками от плоскостопия INNORTO',
  'price': '373 ₽',
  'url': 'https://www.wildberries.ru/catalog/53544024/detail.aspx'},
 {'name': 'Стельки для обуви кроссовок спортивные ортопедические FORTIMET',
  'price': '314 ₽',
  'url': 'https://www.wildberries.ru/catalog/45804581/detail.aspx'},
 {'name': 'Спортивные гелевые стельки INNORTO',
  'price': '503

In [12]:
len(card_list)

1128

Представляем данные в виде датафрейма

In [13]:
import pandas as pd

df = pd.DataFrame(card_list)
df

,name,url,price
0,Стельки для обуви и кроссовок ортопедические м...,https://www.wildberries.ru/catalog/209200599/d...,476 ₽
1,Стельки гелевые женские и мужские INNORTO,https://www.wildberries.ru/catalog/213486771/d...,364 ₽
2,Гелевые стельки для обуви Healthy Feet,https://www.wildberries.ru/catalog/155899851/d...,358 ₽
3,Стельки гелевые спортивные INNORTO,https://www.wildberries.ru/catalog/200025262/d...,467 ₽
4,Стельки гелевые с валиками от плоскостопия INN...,https://www.wildberries.ru/catalog/53544024/de...,373 ₽
...,...,...,...
1123,Стельки,https://www.wildberries.ru/catalog/95854417/de...,919 ₽
1124,Гелевые стельки для обуви Мико,https://www.wildberries.ru/catalog/169797165/d...,388 ₽
1125,Стельки гелевые GO-market,https://www.wildberries.ru/catalog/150961472/d...,422 ₽
1126,Стельки для обуви зимние войлочные MEDICAL PRO,https://www.wildberries.ru/catalog/122557183/d...,397 ₽


запись в csv-файл

In [14]:
df.to_csv('result.csv', index=False)

Вывод:
Данная программа помогает спарсить нужные товары с сайта маркетплейса. Для навигации по сайту я применил функции скролинга (применил функционала JavaScript) и перехода на следующую страницу (по соответствующей кнопке). Во время работы столкнулся с трудностями корректного отображения страницы (после 10 прокрученных страниц появлялась некорректная страница с 1 акционным товаром). Избавился от проблемы путем обновления страницы, иногда многократного обновления. Для более точного работы программы делал некоторые временные паузы с помощью модуля time. Итоговые данные представлены в виде датафрейма и записаны с csv-файл.

PS: дополнительно хочу попробовать реализовать другой способ сбора данных, а именно сначала собрать по страницам список ссылок на товары, а потом в цикле for перейти по ссылке товара и собрать информацию уже в карточке товара. Затем сравнить эти две программы по времени исполнения и затраченной оперативной памяти.